In [1]:
import h5py
import numpy as np
import mmap

In [2]:
data_path = '/data/MEA/neuropixel/'
fname = 'ProcessedSpikes'
ofname = 'ProcessedSpikes.hdf5'

In [3]:
# open/count lines in results file
def mapcount(buf):
    lines = 0
    readline = buf.readline
    while readline():
        lines += 1
    return lines

f = open(data_path+fname, 'r+')
procspikes = mmap.mmap(f.fileno(), 0)
nspikes = mapcount(procspikes)
procspikes.seek(0)
l1 = np.fromstring(procspikes.readline(), sep=' ').astype(float)
cutlen = len(l1)-5
procspikes.seek(0)

In [4]:
print(str(nspikes)+' spikes in file, cutout lengh '+str(cutlen)+' frames')

43006280 spikes in file, cutout lengh 61 frames


In [5]:
#    ch = int(sp[i,0])
#    t = int(sp[i,1])
#    a = sp[i,2]
#    x = sp[i,3]
#    y = sp[i,4]

In [6]:
"""Saves data a hdf5 file.
Offers compression of the shapes, 'lzf'
appears a good trade-off between speed and performance.'"""

compression = 'lzf'

g = h5py.File(data_path+ofname, 'w')

shapes = g.create_dataset("Shapes", shape=(nspikes, cutlen), dtype=int, compression=compression)
sampling = g.create_dataset("Sampling", data=[30000], dtype=int)


In [7]:
%%time

procspikes.seek(0)

batch_size = 500000

data_tmp = np.empty((nspikes, 2))
times_tmp = np.empty(nspikes)
amp_tmp = np.empty(nspikes)
ch_tmp = np.empty(nspikes)
shapes_tmp = np.empty((batch_size, cutlen))

pos = 0
while 1:
    if pos%batch_size==0:
        print(str(np.round(100./nspikes*pos,2))+'%',end='\r')
    l = procspikes.readline()
    if not l : break
    tl = np.fromstring(l, sep=' ').astype(float)
#     if tl.shape[0] is not cutlen+5 : break # use this if a 
    data_tmp[pos, :] = np.array([tl[3], tl[4]]).astype(int)
    times_tmp[pos] = int(tl[1])
    amp_tmp[pos] = int(tl[2])
    ch_tmp[pos] = int(tl[0])
    shapes_tmp[pos%batch_size, :] = tl[5:].astype(int)  
    pos += 1
    if pos%batch_size == 0:
        shapes[(pos//batch_size-1)*batch_size:(pos//batch_size)*batch_size] = shapes_tmp

if pos%batch_size != 0:
    shapes[(pos//batch_size-1)*batch_size:(pos//batch_size-1)*batch_size+pos%batch_size] = shapes_tmp[:pos%batch_size]


CPU times: user 13min 14s, sys: 7.28 s, total: 13min 21s
Wall time: 13min 27s


In [8]:
%%time
ch = g.create_dataset("Channels", data=ch_tmp, dtype=int)
amp = g.create_dataset("Amplitudes", data=amp_tmp, dtype=int)
data = g.create_dataset("Locations", data=data_tmp, dtype=float)
times = g.create_dataset("Times", data=times_tmp, dtype=int)
g.close()

CPU times: user 1.54 s, sys: 2.17 s, total: 3.72 s
Wall time: 13.4 s
